In [1]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
from analysis_functions import Analysis
import sys
sys.path.append("../")
from vqc.vqc_circuits import UQC

First thing I need to do is to be able to create the circuit with the correct weights in pennylane

In [2]:
uqc_2qubits_path = "../../results/uqc_entanglement/2_qubit_uqc_cz"
uqc_4qubits_path = "../../results/uqc_entanglement/4_qubit_uqc_cz"

uqc_2qubits = Analysis(uqc_2qubits_path)
uqc_4qubits = Analysis(uqc_4qubits_path)

Let's try to use negativity to measure the entanglement of the variational quantum circuits

In [3]:
num_qubits = 2
num_layers = 5
entangling_type = None
vqc = UQC(num_qubits, num_layers, entangling_type)

In [4]:
vqc.circuit

(0, 0): ───Rz(x0_0)───Ry(theta0)───Rz(x1_0)───Ry(theta2)───Rz(x2_0)───Ry(theta4)───Rz(x3_0)───Ry(theta6)───Rz(x4_0)───Ry(theta8)───

(0, 1): ───Rz(x0_1)───Ry(theta1)───Rz(x1_1)───Ry(theta3)───Rz(x2_1)───Ry(theta5)───Rz(x3_1)───Ry(theta7)───Rz(x4_1)───Ry(theta9)───

In [5]:
uqc_weights = uqc_2qubits.get_final_weights()
uqc_weights[0]

[array([[0.22377096, 2.4907498 , 0.7647595 , 0.11703885, 2.49186   ,
         0.5816616 , 2.6380851 , 0.5808931 , 0.5977255 , 0.10605836]],
       dtype=float32),
 array([[[-2.3252773 , -0.45936522, -0.05628812,  0.7296369 ],
         [ 0.17676501,  0.06906239,  0.43049112,  1.820161  ]],
 
        [[-0.7617193 , -0.52727914, -0.3803747 , -0.19308297],
         [ 0.01132563,  0.26265895, -0.96870285, -0.8290429 ]],
 
        [[-0.16651799,  0.11960864,  1.7842529 , -0.04611189],
         [ 0.24644308, -0.7475598 , -1.0119281 , -0.12901615]],
 
        [[-0.1548171 ,  0.08413836,  1.8281894 ,  0.6898526 ],
         [-0.6059063 ,  0.6271617 , -1.2207141 ,  0.5451319 ]],
 
        [[-0.10349423,  0.00413808,  1.0261043 , -0.00749924],
         [-0.67405254,  0.7457811 , -0.15448521,  1.2762944 ]]],
       dtype=float32),
 array([[-4.3867836e-03, -2.4723676e-01],
        [-8.9729235e-02,  1.7556483e-01],
        [ 1.7635074e-01, -3.1621188e-01],
        [ 2.3235589e-04,  4.6824500e-01],
  

We dont need the output scaling weights, so lets start by removing them

In [6]:
uqc_weights = uqc_weights[0][:-1]

Let's put the weights in the correct shapes

In [7]:
uqc_weights[0]

array([[0.22377096, 2.4907498 , 0.7647595 , 0.11703885, 2.49186   ,
        0.5816616 , 2.6380851 , 0.5808931 , 0.5977255 , 0.10605836]],
      dtype=float32)

In [8]:
uqc_weights[0] = uqc_weights[0].reshape(num_layers, num_qubits)
uqc_weights

[array([[0.22377096, 2.4907498 ],
        [0.7647595 , 0.11703885],
        [2.49186   , 0.5816616 ],
        [2.6380851 , 0.5808931 ],
        [0.5977255 , 0.10605836]], dtype=float32),
 array([[[-2.3252773 , -0.45936522, -0.05628812,  0.7296369 ],
         [ 0.17676501,  0.06906239,  0.43049112,  1.820161  ]],
 
        [[-0.7617193 , -0.52727914, -0.3803747 , -0.19308297],
         [ 0.01132563,  0.26265895, -0.96870285, -0.8290429 ]],
 
        [[-0.16651799,  0.11960864,  1.7842529 , -0.04611189],
         [ 0.24644308, -0.7475598 , -1.0119281 , -0.12901615]],
 
        [[-0.1548171 ,  0.08413836,  1.8281894 ,  0.6898526 ],
         [-0.6059063 ,  0.6271617 , -1.2207141 ,  0.5451319 ]],
 
        [[-0.10349423,  0.00413808,  1.0261043 , -0.00749924],
         [-0.67405254,  0.7457811 , -0.15448521,  1.2762944 ]]],
       dtype=float32),
 array([[-4.3867836e-03, -2.4723676e-01],
        [-8.9729235e-02,  1.7556483e-01],
        [ 1.7635074e-01, -3.1621188e-01],
        [ 2.3235589e

In [95]:
def uqc_layer(wires, data, rotational_weights, input_weights, bias_weights):
        [qml.RZ(np.dot(2 * input_weights[i], data) + bias_weights[i] , wires[i]) for i in range(len(wires))]
        [qml.RY(2 * rotational_weights[i], wires[i]) for i in range(len(wires))]


def bell_state(params, num_qubits, num_layers, data, qubit_to_measure):
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    return qml.density_matrix(qubit_to_measure)

def ghz_state(params, num_qubits, num_layers, data, qubit_to_measure):
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 3])
    qml.Hadamard(wires=0)
    return qml.density_matrix(qubit_to_measure)

def product_state(params, num_qubits, num_layers, data, qubit_to_measure):
    qml.Hadamard(wires=0)
    qml.Hadamard(wires=1)
    return qml.density_matrix(qubit_to_measure)

def product_state_entropy():
    qml.Hadamard(wires=0)
    qml.Hadamard(wires=1)
    return qml.vn_entropy(wires=0)

def uqc(params, num_qubits, num_layers, data, qubit_to_measure):
        rotational_weights = params[0]
        input_weights = params[1]
        bias_weights = params[2]
        for l in range(num_layers):
            uqc_layer(range(num_qubits), data, rotational_weights[l], input_weights[l], bias_weights[l])
        return qml.density_matrix(qubit_to_measure)

def meyer_wallach(circuit,weights, num_qubits, num_layers, data):
    entropy = 0
    for j in range(num_qubits):
        reduced_density_matrix = circuit(weights, num_qubits, num_layers, data, j)
        trace = np.trace(np.matmul(reduced_density_matrix, reduced_density_matrix))
        entropy += trace
    entropy /= num_qubits
    entropy = 1 - entropy
    return 2*entropy


dev = qml.device("default.qubit", wires = num_qubits)
dev_ghz = qml.device("default.qubit", wires = 4)

circuit = qml.QNode(uqc, dev)
circuit_bell = qml.QNode(bell_state, dev)
circuit_ghz = qml.QNode(ghz_state, dev_ghz)
circuit_product = qml.QNode(product_state, dev)
circuit_product_entropy = qml.QNode(product_state_entropy, dev)

data = np.random.uniform(low = -0.4, high = 0.4, size = (4,))
random_params = [np.random.uniform(low = 0, high = 2*np.pi, size = (num_layers, num_qubits)),
                 np.random.uniform(low = 0, high = 2*np.pi, size = (num_layers, num_qubits, 4)),
                 np.random.uniform(low = 0, high = 2*np.pi, size = (num_layers, num_qubits))]

entropy_bell_state = meyer_wallach(circuit_bell, 0,2, 1, data)
print(entropy_bell_state)

entropy_ghz_state = meyer_wallach(circuit_ghz, 0,4, 1, data)
print(entropy_ghz_state)

entropy_product_state = meyer_wallach(circuit_product, 0,2, 1, data)
print(entropy_product_state)

entropy_uqc = meyer_wallach(circuit, uqc_weights, num_qubits, num_layers, data)
print(entropy_uqc)

entropy_random_uqc = meyer_wallach(circuit, random_params, num_qubits, num_layers, data)
print(entropy_random_uqc)

(1.0000000000000004+0j)
(1.0000000000000009+0j)
(1.7763568394002505e-15+0j)
(4.6629367034256575e-15+0j)
(-1.7763568394002505e-15+0j)
